##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) and [eager execution](https://www.tensorflow.org/guide/eager). The following is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

2024-10-01 15:33:26.133898: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-01 15:33:26.547393: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-01 15:33:26.694717: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-01 15:33:26.739070: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-01 15:33:27.055369: I tensorflow/core/platform/cpu_feature_guar

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [4]:
path_to_file = './docs/utazas_a_holdra.txt'

### Read the data

First, look in the text:

In [5]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 662630 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[:250])

JULES VERNE
UTAZÁS A HOLDBA
______
UTAZÁS A HOLD KÖRÜL
KÉT REGÉNY
FORDÍTOTTA: KILÉNYI MÁRIA
2
TARTALOM
UTAZÁS A HOLDBA
1. A GUN CLUB
2. BARBICANE ELNÖK BEJELENTÉSE
3. A BEJELENTÉS HATÁSA
4. A CAMBRIDGE-I CSILLAGVIZSGÁLÓ VÁLASZA
5. A HOLD REGÉNYE
6. A


In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

118 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation.

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [8]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

I0000 00:00:1727789641.977129   27668 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727789642.264077   27668 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727789642.264112   27668 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727789642.266069   27668 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727789642.266102   27668 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [9]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [10]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[55, 56, 57, 58, 59, 60, 61], [78, 79, 80]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [11]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [12]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings.

In [13]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [14]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [15]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(662630,), dtype=int64, numpy=array([37, 48, 39, ..., 42, 69, 11])>

In [16]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [20]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

J
U
L
E
S
 
V
E
R
N


In [21]:
seq_length = 100


The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [23]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'J' b'U' b'L' b'E' b'S' b' ' b'V' b'E' b'R' b'N' b'E' b'\n' b'U' b'T'
 b'A' b'Z' b'\xc3\x81' b'S' b' ' b'A' b' ' b'H' b'O' b'L' b'D' b'B' b'A'
 b'\n' b'_' b'_' b'_' b'_' b'_' b'_' b'\n' b'U' b'T' b'A' b'Z' b'\xc3\x81'
 b'S' b' ' b'A' b' ' b'H' b'O' b'L' b'D' b' ' b'K' b'\xc3\x96' b'R'
 b'\xc3\x9c' b'L' b'\n' b'K' b'\xc3\x89' b'T' b' ' b'R' b'E' b'G'
 b'\xc3\x89' b'N' b'Y' b'\n' b'F' b'O' b'R' b'D' b'\xc3\x8d' b'T' b'O'
 b'T' b'T' b'A' b':' b' ' b'K' b'I' b'L' b'\xc3\x89' b'N' b'Y' b'I' b' '
 b'M' b'\xc3\x81' b'R' b'I' b'A' b'\n' b'2' b'\n' b'T' b'A' b'R' b'T' b'A'
 b'L' b'O'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [24]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'JULES VERNE\nUTAZ\xc3\x81S A HOLDBA\n______\nUTAZ\xc3\x81S A HOLD K\xc3\x96R\xc3\x9cL\nK\xc3\x89T REG\xc3\x89NY\nFORD\xc3\x8dTOTTA: KIL\xc3\x89NYI M\xc3\x81RIA\n2\nTARTALO'
b'M\nUTAZ\xc3\x81S A HOLDBA\n1. A GUN CLUB\n2. BARBICANE ELN\xc3\x96K BEJELENT\xc3\x89SE\n3. A BEJELENT\xc3\x89S HAT\xc3\x81SA\n4. A CAMBRIDGE-'
b'I CSILLAGVIZSG\xc3\x81L\xc3\x93 V\xc3\x81LASZA\n5. A HOLD REG\xc3\x89NYE\n6. AMIT MINDEN AMERIKAINAK TUDNIA KELL, \xc3\x89S AMIT EGYETLEN '
b'AMERIKAINAK SEM\nSZABAD HINNIE T\xc3\x96BB\xc3\x89\n7. AZ \xc3\x81GY\xc3\x9aGOLY\xc3\x93 HIMNUSZA\n8. AZ \xc3\x81GY\xc3\x9a T\xc3\x96RT\xc3\x89NETE\n9. A L\xc5\x90PORK\xc3\x89RD\xc3\x89S\n10'
b'. HUSZON\xc3\x96TMILLI\xc3\x93 BAR\xc3\x81T \xc3\x89S EGYETLENEGY ELLENS\xc3\x89G\n11. FLORIDA \xc3\x89S TEXAS\n12. URBI ET ORBI\n13. STONE\xe2\x80\x99S HILL'


For training you'll need a dataset of `(input, label)` pairs. Where `input` and
`label` are sequences. At each time step the input is the current character and the label is the next character.

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [25]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [26]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [27]:
dataset = sequences.map(split_input_target)

In [28]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'JULES VERNE\nUTAZ\xc3\x81S A HOLDBA\n______\nUTAZ\xc3\x81S A HOLD K\xc3\x96R\xc3\x9cL\nK\xc3\x89T REG\xc3\x89NY\nFORD\xc3\x8dTOTTA: KIL\xc3\x89NYI M\xc3\x81RIA\n2\nTARTAL'
Target: b'ULES VERNE\nUTAZ\xc3\x81S A HOLDBA\n______\nUTAZ\xc3\x81S A HOLD K\xc3\x96R\xc3\x9cL\nK\xc3\x89T REG\xc3\x89NY\nFORD\xc3\x8dTOTTA: KIL\xc3\x89NYI M\xc3\x81RIA\n2\nTARTALO'


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [29]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)).

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [30]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [55]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(tf.shape(x)[0])
    r = self.gru(x, initial_state=[states], training=training)
    x, states = r[0], r[1:]
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [56]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [59]:
for input_example_batch, target_example_batch in dataset.take(10):
    print(input_example_batch.shape)
    example_batch_predictions = model(input_example_batch, training=True)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100)


2024-10-01 16:05:32.418484: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at cudnn_rnn_ops.cc:1723 : INVALID_ARGUMENT: RNN input_h must be a 3-D vector.


(64, 100, 119) # (batch_size, sequence_length, vocab_size)
(64, 100)


2024-10-01 16:05:32.840830: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at cudnn_rnn_ops.cc:1723 : INVALID_ARGUMENT: RNN input_h must be a 3-D vector.


(64, 100, 119) # (batch_size, sequence_length, vocab_size)
(64, 100)


...

(64, 100, 119) # (batch_size, sequence_length, vocab_size)
(64, 100)


2024-10-01 16:05:35.847328: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at cudnn_rnn_ops.cc:1723 : INVALID_ARGUMENT: RNN input_h must be a 3-D vector.
2024-10-01 16:05:35.847367: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: RNN input_h must be a 3-D vector.
	 [[{{node CudnnRNNV3}}]]


(64, 100, 119) # (batch_size, sequence_length, vocab_size)
(64, 100)


2024-10-01 16:05:36.298755: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at cudnn_rnn_ops.cc:1723 : INVALID_ARGUMENT: RNN input_h must be a 3-D vector.


(64, 100, 119) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [60]:
model.summary()

Model: "my_model_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (64, 100, 256)         │        30,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ ((64, 100, 1024), (64, │     3,938,304 │
│                                 │ 1024))                 │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (64, 100, 119)         │       121,975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,090,743 (15.60 MB)

 Trainable params: 4,090,743 (15.60 MB)

 Non-trainable params: 0 (0.00 B)

To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [76]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [77]:
sampled_indices

array([ 14, 103,  33, 106,  91, 118,  17, 110,  29,  11,  76,   1,  24,
        98,  39, 109,  63,  44,   0,  68,  47, 100,  85,  44, 103,  90,
        56,  11,  97,   1,  35,  39,  85,  96,  52,  91,  31,  50,   3,
        39,  32,  53, 109, 100,  44,  52,  48,  80,  51,  46,   0,  69,
        40, 108,  70, 110,  20,  42,  86,  76,  46,  29,  30,  83,  36,
        46,  28,  39,  59,  96,  52,  73, 103, 110,  73,  65,  16,  79,
        21,  80,  63,   6,  31,  11, 101,  95,   7,  90, 105, 116,  53,
        57,  73,  21,  80,  95,  85,  70, 116,   0])

Decode these to see the text predicted by this untrained model:

In [78]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'lyeket az emberfeletti dolgok\xc3\xa9rt val\xc3\xb3 szenved\xc3\xa9lyes\nlelkesed\xc3\xa9sre \xc3\xb6szt\xc3\xb6k\xc3\xa9l. A birtokl\xc3\xa1si v\xc3\xa1gy, a moh\xc3\xb3 '

Next Char Predictions:
 b'0\xe2\x80\x99F\xe2\x88\x92\xc3\xa9\xef\xa3\xbb3\xef\xa3\xaeB-v\n:\xc5\x90L\xef\xa3\xadiQ[UNK]nT\xc5\xb0\xc3\x93Q\xe2\x80\x99\xc3\xa8b-\xc3\xbc\nHL\xc3\x93\xc3\xbaY\xc3\xa9DW!LEZ\xef\xa3\xad\xc5\xb0QYUzXS[UNK]oM\xef\xa3\xacp\xef\xa3\xae6O\xc3\x96vSBC\xc3\x89ISALe\xc3\xbaYs\xe2\x80\x99\xef\xa3\xaesk2y7zi(D-\xc5\xb1\xc3\xb6)\xc3\xa8\xe2\x80\x9e\xef\xa3\xb9Zcs7z\xc3\xb6\xc3\x93p\xef\xa3\xb9[UNK]'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [40]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [41]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 119)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.77961, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [42]:
tf.exp(example_batch_mean_loss).numpy()

119.05793

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [57]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [45]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [46]:
EPOCHS = 20

In [58]:
import keras

keras.config.disable_traceback_filtering()
keras.backend.set_image_data_format("channels_last")

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20


ValueError: Shape must be rank 3 but is rank 4 for '{{node my_model_5_1/gru_5_1/CudnnRNNV3}} = CudnnRNNV3[T=DT_FLOAT, direction="unidirectional", dropout=0, input_mode="linear_input", is_training=true, num_proj=0, rnn_mode="gru", seed=0, seed2=0, time_major=false](my_model_5_1/embedding_5_1/GatherV2, my_model_5_1/gru_5_1/ExpandDims, my_model_5_1/gru_5_1/CudnnRNNV3/input_c, my_model_5_1/gru_5_1/concat, my_model_5_1/gru_5_1/Fill)' with input shapes: [64,100,256], [1,1,64,1024], [], [3938304], [64].

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(dataset, epochs=1)

Or if you need more control, you can write your own complete custom training loop:

In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))